# Image classification

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/tjAIM7BOYhw?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

画像分類では、画像にラベルまたはクラスを割り当てます。テキストや音声の分類とは異なり、入力は
画像を構成するピクセル値。損傷の検出など、画像分類には多くの用途があります
自然災害の後、作物の健康状態を監視したり、病気の兆候がないか医療画像をスクリーニングしたりするのに役立ちます。

このガイドでは、次の方法を説明します。

1. [Food-101](https://huggingface.co/datasets/food101) データセットの [ViT](https://huggingface.co/docs/transformers/main/ja/tasks/model_doc/vit) を微調整して、画像内の食品を分類します。
2. 微調整したモデルを推論に使用します。

<Tip>

このタスクと互換性のあるすべてのアーキテクチャとチェックポイントを確認するには、[タスクページ](https://huggingface.co/tasks/image-classification) を確認することをお勧めします。

</Tip>

始める前に、必要なライブラリがすべてインストールされていることを確認してください。

```bash
pip install transformers datasets evaluate
```

Hugging Face アカウントにログインして、モデルをアップロードしてコミュニティと共有することをお勧めします。プロンプトが表示されたら、トークンを入力してログインします。

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Load Food-101 dataset

Datasets、🤗 データセット ライブラリから Food-101 データセットの小さいサブセットを読み込みます。これにより、次の機会が得られます
完全なデータセットのトレーニングにさらに時間を費やす前に、実験してすべてが機能することを確認してください。

In [ ]:
from datasets import load_dataset

food = load_dataset("food101", split="train[:5000]")

`train_test_split` メソッドを使用して、データセットの `train` 分割をトレイン セットとテスト セットに分割します。

In [ ]:
food = food.train_test_split(test_size=0.2)

次に、例を見てみましょう。

In [ ]:
food["train"][0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x512 at 0x7F52AFC8AC50>,
 'label': 79}

データセット内の各例には 2 つのフィールドがあります。

- `image`: 食品の PIL 画像
- `label`: 食品のラベルクラス

モデルがラベル ID からラベル名を取得しやすくするために、ラベル名をマップする辞書を作成します。
整数への変換、またはその逆:

In [ ]:
labels = food["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

これで、ラベル ID をラベル名に変換できるようになりました。

In [ ]:
id2label[str(79)]

'prime_rib'

## Preprocess

次のステップでは、ViT 画像プロセッサをロードして画像をテンソルに処理します。

In [ ]:
from transformers import AutoImageProcessor

checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

## Evaluate

トレーニング中にメトリクスを含めると、多くの場合、モデルのパフォーマンスを評価するのに役立ちます。すぐにロードできます
🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) ライブラリを使用した評価方法。このタスクでは、ロードします
[accuracy](https://huggingface.co/spaces/evaluate-metric/accuracy) 指標 (詳細については、🤗 評価 [クイック ツアー](https://huggingface.co/docs/evaluate/a_quick_tour) を参照してくださいメトリクスをロードして計算する方法):

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

次に、予測とラベルを `compute` に渡して精度を計算する関数を作成します。

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

これで `compute_metrics`関数の準備が整いました。トレーニングを設定するときにこの関数に戻ります。

## Train

<Tip>

画像分類用のモデルを微調整する方法の詳細な例については、対応する [PyTorch ノートブック](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/image_classification.ipynb)

</Tip>

## Inference

モデルを微調整したので、それを推論に使用できるようになりました。

推論を実行したい画像を読み込みます。

In [ ]:
ds = load_dataset("food101", split="validation[:10]")
image = ds["image"][0]

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/beignets-task-guide.png" alt="image of beignets"/>
</div>

推論用に微調整されたモデルを試す最も簡単な方法は、それを [pipeline()](https://huggingface.co/docs/transformers/main/ja/main_classes/pipelines#transformers.pipeline) で使用することです。モデルを使用して画像分類用の`pipeline`をインスタンス化し、それに画像を渡します。

In [ ]:
from transformers import pipeline

classifier = pipeline("image-classification", model="my_awesome_food_model")
classifier(image)

[{'score': 0.31856709718704224, 'label': 'beignets'},
 {'score': 0.015232225880026817, 'label': 'bruschetta'},
 {'score': 0.01519392803311348, 'label': 'chicken_wings'},
 {'score': 0.013022331520915031, 'label': 'pork_chop'},
 {'score': 0.012728818692266941, 'label': 'prime_rib'}]

必要に応じて、`pipeline`の結果を手動で複製することもできます。